# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Nov 14 2022 9:25AM,250659,10,Enova-10961,Emerginnova,Released
1,Nov 14 2022 9:25AM,250659,10,Enova-10962,Emerginnova,Released
2,Nov 14 2022 9:25AM,250659,10,Enova-10963,Emerginnova,Released
3,Nov 14 2022 9:25AM,250659,10,DNMD-26455,Emerginnova,Released
4,Nov 14 2022 9:25AM,250659,10,Enova-10964,Emerginnova,Released
5,Nov 14 2022 9:25AM,250659,10,Eme-105801,Emerginnova,Released
6,Nov 14 2022 9:11AM,250657,10,8763000,Yusen – 3D Matrix,Executing
7,Nov 14 2022 9:08AM,250656,20,ATR-I000022,"HVL, LLC dba Atrium Innovations",Released
8,Nov 14 2022 9:01AM,250651,10,0086165617,ISDIN Corporation,Released
9,Nov 14 2022 9:01AM,250651,10,0086165616,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
15,250655,Executing,1
16,250655,Released,3
17,250656,Released,1
18,250657,Executing,1
19,250659,Released,6


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
250654,12.0,36.0
250655,1.0,3.0
250656,NaN,1.0
250657,1.0,NaN
250659,NaN,6.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
250634,0.0,2.0
250635,41.0,19.0
250638,0.0,5.0
250644,1.0,1.0
250645,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
250634,0,2
250635,41,19
250638,0,5
250644,1,1
250645,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,250634,0,2
1,250635,41,19
2,250638,0,5
3,250644,1,1
4,250645,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,250634,,2
1,250635,41,19
2,250638,,5
3,250644,1,1
4,250645,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Nov 14 2022 9:25AM,250659,10,Emerginnova
6,Nov 14 2022 9:11AM,250657,10,Yusen – 3D Matrix
7,Nov 14 2022 9:08AM,250656,20,"HVL, LLC dba Atrium Innovations"
8,Nov 14 2022 9:01AM,250651,10,ISDIN Corporation
72,Nov 14 2022 9:00AM,250654,12,Hush Hush
120,Nov 14 2022 8:59AM,250655,10,"Snap Medical Industries, LLC"
124,Nov 14 2022 8:54AM,250652,19,"GCH Granules USA, Inc."
125,Nov 14 2022 8:46AM,250649,19,"Avet Pharmaceuticals Labs, Inc"
126,Nov 14 2022 8:44AM,250648,19,"Emersa Waterbox, LLC"
127,Nov 14 2022 8:40AM,250647,19,ACG North America LLC


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Nov 14 2022 9:25AM,250659,10,Emerginnova,,6
1,Nov 14 2022 9:11AM,250657,10,Yusen – 3D Matrix,1,
2,Nov 14 2022 9:08AM,250656,20,"HVL, LLC dba Atrium Innovations",,1
3,Nov 14 2022 9:01AM,250651,10,ISDIN Corporation,,64
4,Nov 14 2022 9:00AM,250654,12,Hush Hush,12,36
5,Nov 14 2022 8:59AM,250655,10,"Snap Medical Industries, LLC",1,3
6,Nov 14 2022 8:54AM,250652,19,"GCH Granules USA, Inc.",,1
7,Nov 14 2022 8:46AM,250649,19,"Avet Pharmaceuticals Labs, Inc",,1
8,Nov 14 2022 8:44AM,250648,19,"Emersa Waterbox, LLC",,1
9,Nov 14 2022 8:40AM,250647,19,ACG North America LLC,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Nov 14 2022 9:25AM,250659,10,Emerginnova,6,
1,Nov 14 2022 9:11AM,250657,10,Yusen – 3D Matrix,,1
2,Nov 14 2022 9:08AM,250656,20,"HVL, LLC dba Atrium Innovations",1,
3,Nov 14 2022 9:01AM,250651,10,ISDIN Corporation,64,
4,Nov 14 2022 9:00AM,250654,12,Hush Hush,36,12
5,Nov 14 2022 8:59AM,250655,10,"Snap Medical Industries, LLC",3,1
6,Nov 14 2022 8:54AM,250652,19,"GCH Granules USA, Inc.",1,
7,Nov 14 2022 8:46AM,250649,19,"Avet Pharmaceuticals Labs, Inc",1,
8,Nov 14 2022 8:44AM,250648,19,"Emersa Waterbox, LLC",1,
9,Nov 14 2022 8:40AM,250647,19,ACG North America LLC,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Nov 14 2022 9:25AM,250659,10,Emerginnova,6,
1,Nov 14 2022 9:11AM,250657,10,Yusen – 3D Matrix,,1
2,Nov 14 2022 9:08AM,250656,20,"HVL, LLC dba Atrium Innovations",1,
3,Nov 14 2022 9:01AM,250651,10,ISDIN Corporation,64,
4,Nov 14 2022 9:00AM,250654,12,Hush Hush,36,12


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Nov 14 2022 9:25AM,250659,10,Emerginnova,6.0,NaN
1,Nov 14 2022 9:11AM,250657,10,Yusen – 3D Matrix,NaN,1.0
2,Nov 14 2022 9:08AM,250656,20,"HVL, LLC dba Atrium Innovations",1.0,NaN
3,Nov 14 2022 9:01AM,250651,10,ISDIN Corporation,64.0,NaN
4,Nov 14 2022 9:00AM,250654,12,Hush Hush,36.0,12.0


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Nov 14 2022 9:25AM,250659,10,Emerginnova,6.0,0.0
1,Nov 14 2022 9:11AM,250657,10,Yusen – 3D Matrix,0.0,1.0
2,Nov 14 2022 9:08AM,250656,20,"HVL, LLC dba Atrium Innovations",1.0,0.0
3,Nov 14 2022 9:01AM,250651,10,ISDIN Corporation,64.0,0.0
4,Nov 14 2022 9:00AM,250654,12,Hush Hush,36.0,12.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2005174,100.0,43.0
12,250654,36.0,12.0
19,1503886,7.0,1.0
20,250656,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2005174,100.0,43.0
1,12,250654,36.0,12.0
2,19,1503886,7.0,1.0
3,20,250656,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,100.0,43.0
1,12,36.0,12.0
2,19,7.0,1.0
3,20,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,100.0
1,12,Released,36.0
2,19,Released,7.0
3,20,Released,1.0
4,10,Executing,43.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,19,20
Status,,,,
Executing,43.0,12.0,1.0,0.0
Released,100.0,36.0,7.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,19,20
0,Executing,43.0,12.0,1.0,0.0
1,Released,100.0,36.0,7.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,19,20
0,Executing,43.0,12.0,1.0,0.0
1,Released,100.0,36.0,7.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()